<a href="https://colab.research.google.com/github/Vaibhavs10/scratchpad/blob/main/endpoints_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from huggingface_hub import list_inference_endpoints

In [2]:
NAMESPACE = "hf-inference"
ENDPOINT_PREFIX = "auto-"
HF_TOKEN = "..."

In [5]:
endpoints = list_inference_endpoints(namespace=NAMESPACE)
# Extract model names from endpoints starting with our prefix
deployed_models = []
actual_names = []
for endpoint in endpoints:
    if endpoint.name.startswith(ENDPOINT_PREFIX):
        # Extract the model name from the repository field
        deployed_models.append(endpoint.repository)
        actual_names.append(endpoint.name)

In [10]:
import subprocess
import json

def update_endpoint(actual_name, token):
    """Updates a Hugging Face inference endpoint."""
    try:
        payload = {
            "compute": {
                "scaling": {
                    "minReplica": 1,
                    "scaleToZeroTimeout": None
                }
            }
        }

        command = [
            "curl",
            "-X", "PUT",
            f"https://api.endpoints.huggingface.cloud/v2/endpoint/hf-inference/{actual_name}",
            "-H", "accept: application/json",
            "-H", f"authorization: Bearer {token}",
            "-H", "content-type: application/json",
            "-d", json.dumps(payload)
        ]

        process = subprocess.run(command, capture_output=True, text=True, check=True)
        print(f"Successfully updated endpoint {actual_name}: {process.stdout}")
    except subprocess.CalledProcessError as e:
        print(f"Error updating endpoint {actual_name}: {e.stderr}")

In [12]:
for actual_name in actual_names:
  update_endpoint(actual_name, token=HF_TOKEN)

Successfully updated endpoint auto-distilbart-mnli-12-6: {"name":"auto-distilbart-mnli-12-6","type":"protected","provider":{"vendor":"aws","region":"us-east-1"},"compute":{"id":"aws-us-east-1-intel-spr-x2","accelerator":"cpu","instanceType":"intel-spr","instanceSize":"x2","scaling":{"minReplica":1,"maxReplica":1,"metric":"hardwareUsage","measure":{"hardwareUsage":null}}},"experimentalFeatures":{"cacheHttpResponses":false,"kvRouter":null},"model":{"repository":"valhalla/distilbart-mnli-12-6","revision":"f14383bf830237f338e7d597955f156590fddc2e","task":"zero-shot-classification","framework":"pytorch","image":{"huggingface":{}},"secrets":{},"env":{}},"status":{"createdAt":"2025-05-05T12:03:29.021Z","createdBy":{"id":"61b85ce86eb1f2c5e6233736","name":"reach-vb"},"updatedAt":"2025-05-08T09:06:46.277Z","updatedBy":{"id":"61b85ce86eb1f2c5e6233736","name":"reach-vb"},"state":"initializing","message":"No replicas are ready yet","readyReplica":0,"targetReplica":1},"route":{"domain":"api-inferenc